In [1]:
import numpy as np
import torch
from tsGaussian.torch_tsgaussian import TangentSpaceGaussian
from stable_baselines_utils import TangentSpaceGaussian as TSG
# from pytorch3d.transforms.so3 import (
#     so3_exp_map,
#     so3_relative_angle,
# )

/home/fantasticoven/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tg = TangentSpaceGaussian(None)

# Test liegroup torch

In [3]:
from liegroups.torch import SO3

In [4]:
C = SO3.exp(torch.Tensor([[1,2,3],
                         [0,0,0]]))
C

<liegroups.torch.so3.SO3Matrix>
| tensor([[[-0.6949,  0.7135,  0.0893],
|          [-0.1920, -0.3038,  0.9332],
|          [ 0.6930,  0.6313,  0.3481]],
| 
|         [[ 1.0000,  0.0000,  0.0000],
|          [ 0.0000,  1.0000,  0.0000],
|          [ 0.0000,  0.0000,  1.0000]]])

# Test torch_tsgaussian sample

In [5]:
R_mu = torch.eye(3).reshape((1,3,3))
sigma = torch.ones(3).reshape((1,3))

In [6]:
R_quat, R_x = tg.rsample(R_mu, sigma)

sigma:  tensor([[1., 1., 1.]])
torch.Size([1, 3, 3])


In [7]:
torch.bmm(torch.transpose(R_x, 1, 2), R_x)

tensor([[[1.0000e+00, 2.9802e-08, 0.0000e+00],
         [2.9802e-08, 1.0000e+00, 2.9802e-08],
         [0.0000e+00, 2.9802e-08, 1.0000e+00]]])

# Test torch_tsgaussian normal_term

In [8]:
sigma = torch.ones(3).reshape((1,3))
sigma

tensor([[1., 1., 1.]])

In [9]:
tg.normal_term(sigma)

tensor([15.7496])

# Test torch_tsgaussian log_map

In [10]:
R_1 = torch.eye(3).reshape((1, 3, 3))
R_2 = torch.eye(3).reshape((1, 3, 3))

In [11]:
tg.log_map(R_1, R_2)

tensor([0., 0., 0.])

# Test torch_tsgaussian log_probs

In [12]:
R_x = torch.eye(3).reshape((1,3,3))
R_mu = torch.zeros(3,3).reshape((1,3,3))
R_x = R_x.repeat(5, 1, 1)
R_mu = R_mu.repeat(5, 1, 1)
sigma = torch.ones(3).reshape((1,3))

In [13]:
# tg.log_probs(R_x, R_mu, sigma)

In [14]:
np.e ** (-2.7568)

0.06349462641817973

all codes run for torch_tsgaussian now, need to check it's correctness and make it into batch version.

# Test TangentSpaceGaussian actions_from_params

In [15]:
tsg = TSG(None)

In [16]:
print(tsg.distribution)

In [17]:
tsg

In [18]:
tsg.actions_from_params(torch.eye(3).reshape((1,3,3)), torch.ones(3).reshape((1,3)))

sigma:  tensor([[1., 1., 1.]])
torch.Size([1, 3, 3])


(tensor([[-0.2309, -0.1942,  0.2587,  0.9176]]),
 tensor([[[ 0.7908, -0.3850, -0.4759],
          [ 0.5644,  0.7595,  0.3233],
          [ 0.2370, -0.5243,  0.8179]]]))

# Test TangentSpaceGaussian log_prob_from_params

In [19]:
torch.eye(3).repeat(2,1,1).size()

torch.Size([2, 3, 3])

In [20]:
torch.ones(3).repeat(2,1).size()

torch.Size([2, 3])

In [21]:
# tsg.log_prob_from_params(torch.eye(3).repeat(2,1,1), torch.ones(3))

In [22]:
x = torch.randn(2, 4, 4)
y = torch.linalg.inv(x)
y

tensor([[[ 0.8972,  0.3311, -2.0802,  0.1954],
         [-0.4544,  0.0150, -0.1455,  0.1318],
         [-0.1425,  0.6359, -1.6084, -1.3160],
         [ 0.6918, -0.0042,  3.0271,  1.8034]],

        [[-0.2175, -0.6450, -0.9562,  0.3594],
         [ 2.2492,  0.1025,  2.0803, -0.1886],
         [ 1.8147, -0.1330,  2.0608, -0.5000],
         [-1.9430, -0.4256, -1.6181,  0.7988]]])

Again, codes can run, but need to check correctness.

# Try to run training

In [23]:
import torch
from absl import app, flags
from stable_baselines3 import SAC, PPO
from envs.wahba import Wahba
from stable_baselines_utils import CustomSACPolicy, \
    CustomCNN

In [26]:
def main(argv):
    env = Wahba()
    device = torch.device('cpu')
    policy_kwargs = dict(
        features_extractor_class = CustomCNN,
        features_extractor_kwargs = dict(features_dim = 256))
    policy_kwargs['n_critics'] = 1
    policy_kwargs['share_features_extractor'] = False
    policy = CustomSACPolicy
    model = SAC(policy, env, verbose = 1, ent_coef = 'auto_0.1',
                policy_kwargs = policy_kwargs, device = device)
    model.learn(total_timesteps = 500, eval_freq = 100, n_eval_episodes = 100)

In [27]:
from torch import autograd
with autograd.detect_anomaly():
    main(None)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -5.48    |
| time/              |          |
|    episodes        | 4        |
|    fps             | 800      |
|    time_elapsed    | 0        |
|    total_timesteps | 4        |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -6.01    |
| time/              |          |
|    episodes        | 8        |
|    fps             | 826      |
|    time_elapsed    | 0        |
|    total_timesteps | 8        |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -5.91    |
| time/              |          |
|    episodes        | 12       |
|    fps             |

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -6.09    |
| time/              |          |
|    episodes        | 100      |
|    fps             | 832      |
|    time_elapsed    | 0        |
|    total_timesteps | 100      |
---------------------------------


/tmp/ipykernel_6198/381243426.py:2: UserWarning: Anomaly Detection has been enabled. This mode will increase the runtime and should only be enabled for debugging.
  with autograd.detect_anomaly():


tensor([[ 0.0090, -0.0270,  0.0588,  0.0068, -0.0629, -0.0564, -0.0135, -0.0556,
          0.0520, -0.0301,  0.0054,  0.0176]])
sigma:  tensor([[0.0090, 0.0270, 0.0588]])
torch.Size([1, 3, 3])
tensor([[ 0.0088, -0.0280,  0.0590,  ..., -0.0309,  0.0059,  0.0176],
        [ 0.0089, -0.0279,  0.0591,  ..., -0.0309,  0.0058,  0.0174],
        [ 0.0089, -0.0271,  0.0588,  ..., -0.0302,  0.0050,  0.0177],
        ...,
        [ 0.0088, -0.0282,  0.0590,  ..., -0.0306,  0.0055,  0.0179],
        [ 0.0091, -0.0271,  0.0587,  ..., -0.0302,  0.0053,  0.0176],
        [ 0.0089, -0.0275,  0.0588,  ..., -0.0307,  0.0052,  0.0173]],
       grad_fn=<AddmmBackward>)
sigma:  tensor([[0.0088, 0.0280, 0.0590],
        [0.0089, 0.0279, 0.0591],
        [0.0089, 0.0271, 0.0588],
        [0.0087, 0.0276, 0.0588],
        [0.0090, 0.0276, 0.0589],
        [0.0088, 0.0269, 0.0588],
        [0.0089, 0.0272, 0.0586],
        [0.0088, 0.0271, 0.0586],
        [0.0088, 0.0269, 0.0588],
        [0.0088, 0.0276, 0.

torch.Size([256, 3, 3])
actions:  tensor([[ 0.7409, -0.3898, -0.5165, -0.1801],
        [ 0.7498, -0.4128, -0.4879, -0.1713],
        [ 0.7716, -0.3797, -0.4790, -0.1760],
        ...,
        [ 0.7458, -0.3936, -0.5081, -0.1750],
        [ 0.7563, -0.3878, -0.4989, -0.1692],
        [ 0.7554, -0.3983, -0.4927, -0.1669]])
actions_mat:  tensor([[[ 0.1627, -0.7636, -0.6249],
         [-0.3915, -0.6313,  0.6695],
         [-0.9057,  0.1357, -0.4016]],

        [[ 0.1831, -0.7862, -0.5902],
         [-0.4519, -0.6004,  0.6597],
         [-0.8731,  0.1459, -0.4653]],

        [[ 0.2527, -0.7546, -0.6056],
         [-0.4174, -0.6497,  0.6354],
         [-0.8729,  0.0922, -0.4791]],

        ...,

        [[ 0.1738, -0.7650, -0.6202],
         [-0.4092, -0.6289,  0.6611],
         [-0.8957,  0.1389, -0.4223]],

        [[ 0.2014, -0.7554, -0.6235],
         [-0.4178, -0.6420,  0.6429],
         [-0.8860,  0.1310, -0.4449]],

        [[ 0.1971, -0.7663, -0.6115],
         [-0.4373, -0.6270,  0

...

tensor([[ 5.6508, -5.0040,  5.4816, -0.0631,  0.0160, -1.0440, -0.2049,  0.5480,
         -1.2999, -0.6847, -0.4545,  0.0737]])
sigma:  tensor([[5.6508, 5.0040, 5.4816]])
torch.Size([1, 3, 3])
tensor([[ 5.5719, -4.9361,  5.4059,  ..., -0.6752, -0.4476,  0.0731],
        [ 5.4752, -4.8506,  5.3131,  ..., -0.6650, -0.4402,  0.0715],
        [ 5.4768, -4.8515,  5.3142,  ..., -0.6646, -0.4402,  0.0713],
        ...,
        [ 5.4739, -4.8497,  5.3121,  ..., -0.6646, -0.4400,  0.0717],
        [ 5.5721, -4.9362,  5.4060,  ..., -0.6752, -0.4476,  0.0731],
        [ 5.3371, -4.7293,  5.1800,  ..., -0.6488, -0.4285,  0.0696]],
       grad_fn=<AddmmBackward>)
sigma:  tensor([[5.5719, 4.9361, 5.4059],
        [5.4752, 4.8506, 5.3131],
        [5.4768, 4.8515, 5.3142],
        [5.5721, 4.9359, 5.4060],
        [5.5859, 4.9483, 5.4193],
        [5.5302, 4.8994, 5.3659],
        [5.5708, 4.9349, 5.4047],
        [5.3147, 4.7090, 5.1589],
        [5.5843, 4.9465, 5.4175],
        [5.5126, 4.8826, 5.

tensor([[ 6.8052, -6.0386,  6.5966, -0.0408,  0.0326, -1.2606, -0.2406,  0.6854,
         -1.5847, -0.8512, -0.5399,  0.1338]])
sigma:  tensor([[6.8052, 6.0386, 6.5966]])
torch.Size([1, 3, 3])
tensor([[ 6.8777, -6.1028,  6.6659,  ..., -0.8597, -0.5459,  0.1357],
        [ 6.9521, -6.1677,  6.7380,  ..., -0.8695, -0.5521,  0.1366],
        [ 6.8533, -6.0811,  6.6424,  ..., -0.8572, -0.5436,  0.1348],
        ...,
        [ 6.5253, -5.7912,  6.3272,  ..., -0.8184, -0.5175,  0.1275],
        [ 6.8418, -6.0700,  6.6317,  ..., -0.8565, -0.5434,  0.1340],
        [ 6.8577, -6.0851,  6.6466,  ..., -0.8574, -0.5443,  0.1353]],
       grad_fn=<AddmmBackward>)
sigma:  tensor([[6.8777, 6.1028, 6.6659],
        [6.9521, 6.1677, 6.7380],
        [6.8533, 6.0811, 6.6424],
        [6.8424, 6.0706, 6.6323],
        [6.8117, 6.0441, 6.6023],
        [6.7682, 6.0058, 6.5614],
        [6.7716, 6.0089, 6.5636],
        [6.8721, 6.0978, 6.6605],
        [6.9978, 6.2077, 6.7823],
        [6.7746, 6.0116, 6.

tensor([[ 8.2103, -7.3000,  7.9544, -0.0166,  0.0669, -1.5206, -0.2913,  0.8597,
         -1.9318, -1.0502, -0.6268,  0.2157]])
sigma:  tensor([[8.2103, 7.3000, 7.9544]])
torch.Size([1, 3, 3])
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -6.01    |
| time/              |          |
|    episodes        | 128      |
|    fps             | 6        |
|    time_elapsed    | 20       |
|    total_timesteps | 128      |
| train/             |          |
|    actor_loss      | 5.45     |
|    critic_loss     | 2.11     |
|    ent_coef        | 0.0992   |
|    ent_coef_loss   | -20.9    |
|    learning_rate   | 0.0003   |
|    n_updates       | 27       |
---------------------------------
tensor([[ 8.3789, -7.4495,  8.1163,  ..., -1.0712, -0.6394,  0.2205],
        [ 8.3693, -7.4413,  8.1075,  ..., -1.0696, -0.6386,  0.2205],
        [ 8.3927, -7.4619,  8.1298,  ..., -1.0727, -0.6404,  0.2210],
        ...,
      

/home/fantasticoven/.local/lib/python3.8/site-packages/torch/autograd/__init__.py:145: UserWarning: Error detected in MulBackward0. Traceback of forward call that caused the error:
  File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/fantasticoven/.local/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/home/fantasticoven/.local/lib/python3.8/site-packages/traitlets/config/application.py", line 846, in launch_instance
    app.start()
  File "/home/fantasticoven/.local/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 707, in start
    self.io_loop.start()
  File "/home/fantasticoven/.local/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 199, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.8/asyncio/base_events.py", l

RuntimeError: Function 'MulBackward0' returned nan values in its 1th output.

In [ ]:
%load_ext tensorboard
import tensorflow as tf
import numpy as np
import datetime
import matplotlib.pyplot as plt
from tensorboard.backend.event_processing import event_accumulator

In [ ]:
%tensorboard --logdir ./sac

# Experiments for batch operations

In [ ]:
sigma = torch.ones(1, 3)
omiga = torch.normal(torch.zeros(1, 3), sigma)
omiga

In [ ]:
def transfer(omiga):
    omiga_0, omiga_1, omiga_2 = omiga[0], omiga[1], omiga[2]
    omiga_hat = torch.tensor([[0, -omiga_2, omiga_1],
                                [omiga_2, 0, -omiga_0],
                                [-omiga_1, omiga_0, 0]])
    return omiga_hat

In [ ]:
from functorch import vmap
batch_transfer = vmap(transfer)
batch_transfer(omiga)

In [ ]:
from liegroups.torch import SO3
C = SO3.exp(torch.Tensor([[1,2,3],
                          [0,0,0]]))
print(torch.Tensor([[1,2,3],
                          [0,0,0]]).size())
SO3.log(C)

In [ ]:
np.log(1)

# Question to ask: the original wahba problem action is (4,), in our case actions are (3,3).